In [ ]:
from bs4 import BeautifulSoup
import requests
import argparse
import csv
import concurrent.futures

In [ ]:
def prepare_soup(url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'}
    try:
        response =  requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        return soup
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
        exit()
    except Exception as e:
        print(f'Something went wrong...Error: {e}') 
            exit()

In [1]:
def write_table(table, filename, mode, cell_val=[], link=[]):
    with open(filename, mode, encoding='utf-8') as f:
        writer = csv.writer(f)
        for tr in table.findAll('tr')[1:]:
            row  = []
            for data in tr.findAll(['th', 'td']):
                row.append(data.get_text())
            writer.writerow(row)

def write_list(ul, writer, cell_val, link):
    string = str(ul.text).replace("\n", " | ")
    writer.writerow([cell_val, link, string])

# Main

In [5]:
# url = "http://www.governotransparente.com.br/transparencia/4382489/consultarliqdesporc/resultado?ano=8&inicio=01%2F01%2F2021&fim=24%2F01%2F2021&orgao=-1&elem=-1&unid=-1&valormax=&valormin=&credor=-1&clean=false"
# name = "consultarliqdesporc"

In [6]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('name', type=str, help='Name to csv file')
    parser.add_argument('url', type=str, help='url of the page')
    args = parser.parse_args()
    url = args.url
    name = args.name
    filename = name+'.csv'
    soup = prepare_soup(url)
    main_table = soup.find('table')
    write_table(main_table, filename, 'w+')

In [7]:
head_url = 'http://www.governotransparente.com.br'
empenho = []
documento = []
links = main_table.find_all('a')
for link in links:
    category = link['href'].split('/')[3]
    if category == 'detalharempenhoportal':
        empenho.append((link.text, head_url + link['href']))
    else:
        documento.append((link.text, head_url+link['href']))

# emp

In [8]:
def write_inner_table(table, writer,val, url):
    table = container.find('table')
    for tr in table.findAll('tr')[1:]:
        row  = []
        for data in tr.findAll(['td']):
            row.append(data.get_text())
        writer.writerow([val, url] + row)
        

In [9]:
def thread_request(x):
    val, url = x
    response = requests.get(url)
    return (val, url, response)

In [10]:
def concurrent_response(arr):
    POOL_SIZE = 4
    with concurrent.futures.ThreadPoolExecutor(max_workers=POOL_SIZE) as executor:
        response_arr = executor.map(thread_request, [x for x in arr[:3]])#######################
    response_arr = list(response_arr)
    return response_arr

In [11]:
def create_files_empheno(name):
    extras = ["Cell value", "Link"]
    f1 = open(f'{name}.empheno.Destalhes do empenho.csv', "w+", encoding='utf-8')
    writer1 = csv.writer(f1)
    writer1.writerow(extras + ["List items"])
    f2 = open(f'{name}.empheno.Movimentos do empenho.csv', "w+", encoding='utf-8')
    writer2 = csv.writer(f2)
    writer2.writerow(extras + ['Documento', 'Data', 'Tipo', 'Registro', 'Valor (R$)'])
    return f1, f2, writer1, writer2
response_arr = concurrent_response(empenho)

In [14]:
f1, f2, w1, w2 = create_files_empheno(name)
for val, url, response in response_arr:
    soup = BeautifulSoup(response.text, "html.parser")
    container = soup.find_all('div', class_='row clearfix')[1]
    ul = container.find('ul')
    write_list(ul, w1, val, url)
    table = container.find('table')
    write_inner_table(table, w2, val, url)
f1.close()
f2.close()

In [15]:
def create_files_documento(name):
    extras = ["Cell value", "Link"]
    f1 = open(f'{name}.documento.Dados da liquidação.csv', "w+", encoding='utf-8')
    w1 = csv.writer(f1)
    w1.writerow(extras + ["List items"])
    f2 = open(f'{name}.documento.Dados do pagamento do empenho.csv', "w+", encoding='utf-8')
    w2 = csv.writer(f2)
    w2.writerow(extras + ["Tipo", "Forma", "Valor", "Documento", "Banco", "Nome da agência", "Número da agência", "Conta corrente"])
    f3 = open(f'{name}.documento.Dados do empenho.csv', "w+", encoding='utf-8')
    w3 = csv.writer(f3)
    w3.writerow(extras + ["List items"])
    f4 = open(f'{name}.documento.Movimentos do empenho.csv', "w+", encoding='utf-8')
    w4 = csv.writer(f4)
    w4.writerow(extras + ['Documento', 'Data', 'Tipo', 'Registro', 'Valor (R$)'])
    return [f1, f2, f3, f4], w1, w2, w3, w4
response_arr = concurrent_response(documento)

In [16]:
f, w1, w2, w3, w4 = create_files_documento(name)
for val, url, response in response_arr:
    soup = BeautifulSoup(response.text, "html.parser")
    container = soup.find_all('div', class_='row clearfix')[1]
    ul1 = container.find_all('ul')[0]
    ul2 = container.find_all('ul')[1]
    table1 = container.find_all('table')[0]
    table2 = container.find_all('table')[1]
    write_list(ul1, w1, val, url)
    write_inner_table(table1, w2, val, url)
    write_list(ul2, w3, val, url)
    write_inner_table(table2, w4, val, url)
for file in f:
    file.close()

In [14]:
import requests
response = requests.get('http://www.goog.com', headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'})
print(response.status_code)

200
